In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
data = pd.read_csv("properati_data_fixed_all_years_duplicated_with_outliers.csv", compression = "gzip")

In [ ]:
posibles_descripciones_ambientes = {"1 ambiente": 1, "monoambiente": 1, "mono ambiente":1, "2 ambientes": 2,"3 ambientes": 3,
                          "4 ambientes": 4,"5 ambientes": 5, "6 ambientes": 6, "7 ambientes": 7,"8 ambientes": 8,
                          "9 ambientes": 1, "10 ambientes": 10,
                         "ambientes: 1": 1, "ambientes: 2": 2, "ambientes: 3": 3, "ambientes: 4": 4, "ambientes: 5": 5,
                         "ambientes: 6": 6, "ambientes: 7": 7, "un ambiente": 1, "dos ambientes": 2, "tres ambientes": 3,
                         "cuatro ambientes": 4,"cinco ambientes": 5," seis ambientes": 6, "siete ambientes": 7,
                         "2 impecables ambientes": 2, "3 impecables ambientes": 3, "4 amplios ambientes": 4, "2 amplios ambientes": 2,
                          "3 amplios ambientes": 3, "5 amplios ambientes": 5, "2  ambientes": 2, "3  ambientes": 3,
                          "2 grandes ambientes": 2, "6 amplios ambientes": 6, "1 ambientes": 1
                         ,"3 amplios  ambientes": 3, "3 y ½ ambientes": 3.5,"2 y ½ ambientes": 2.5, "2 y medio ambiente": 2.5,
                         "(2) ambientes":2, "2 amplisimos ambientes": 2,"4  ambientes":4,"3 grandes ambientes":3,
                                   "4 ½ ambientes": 4.5, "4 cÃ²modos y confortables ambientes":4, "cantidad de ambientes 4":4,
                                   "cantidad de ambientes 3": 3,"ambientes : 4":4,"ambientes : 5":5,"ambientes : 3":3,"ambientes : 2":2,
                                   "ambientes : 1":1,"chalet de 5  ambientes":5,"4 cÃ³modos y confortables ambientes":4,"4   ambientes":4,
                                   "3 buenos ambientes":3,"4 còmodos y confortables ambientes":4,"2 amb.":2,"3 amb.":3,"4 amb.":4,"1 amb.":1, "5 amb.":5,"6 amb.":6,"7 amb.":7,
                                   "3 (tres) ambientes": 3, "2 (dos) ambientes":2, "4 (cuatro) ambientes":4, "5 (cinco) ambientes": 5, "1 (uno) ambientes": 1, "1 (uno) ambiente":1,
                                   "6 (seis) ambientes":6, "7 (siete) ambientes":7, "8 (ocho) ambientes":8, "9 (nueve) ambientes":9, "10 (diez) ambientes":10, "1 amb":1, "2 amb":2,
                                   "3 amb":3, "4 amb":4, "5 amb":5, "6 amb":6,"7 amb":7,"8 amb":8,"2 ambiente":2,"3 ambiente":3,"4 ambiente":4, "1amb":1,"2amb":2,"3amb":3,"4amb":4,
                                   "5amb":5,"6amb":6,"8amb":8,"7amb":7}


In [ ]:

def fix_ambientes(row, search_title, search_extra, search_description):
    
    if pd.isnull(row["rooms"]):
        return np.nan#Ya fueron fixeadas
    cols = []
    if search_description:
        descripcion = row["description"]
        cols.append(descripcion)
    if search_extra:
        extra = row["extra"]
        cols.append(extra)
    if search_title:
        title = row["title"]
        cols.append(title)
        
    ambiente_actual = row["rooms"]
    for col in cols:
        if not(pd.isnull(col)):
            for k in posibles_descripciones_ambientes:
        
                if k in col:
                    value_found = float(posibles_descripciones_ambientes[k])
                    if float(row["rooms"]) != value_found:
                        return value_found
    return row["rooms"]

In [ ]:
data_2017 = data[data["year"] == 2017]

In [ ]:
#Sacamos las de Zona Norte porque no tenemos que predecir casi ninguna de esa zona.
data_2017_zo_zs_caba = data_2017[(data_2017["state_name"] == "Capital Federal") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Sur") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Oeste")]

In [ ]:
#Antes vuelvo a aplicar esto xq habia algunos que se pueden corregir
data_2017_zo_zs_caba["rooms"] = data_2017_zo_zs_caba.apply(lambda row: fix_ambientes(row, False, False, True), axis = 1)

In [ ]:
#Los que tienen nulo (room, surface total), (room, cubierta), (cubierta, total) no nos sirven demasiado
len(data_2017_zo_zs_caba)

In [ ]:
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)

In [ ]:
data_2017_zo_zs_caba.isnull().sum()

In [ ]:
#A los de rooms nulas tratamos de rellenarlas con con los que tengan superficie total y cubierta parecida (su promedio)

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"].isnull()]["surface_covered_in_m2"].values[4]

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values[3]

In [ ]:
data_2017_zo_zs_caba["rooms"].value_counts()

In [ ]:
#Con esto vemos que ya mas de 8 ambientes es algo raro, 

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 11]["description"].values[11]

In [ ]:
#Pongo en NaN aquellos que tengan mas de 8 ambientes

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values

In [ ]:
#En promedio por ambiente, cual es su superficie cubierta?
ambs = range(1,9)
for amb in ambs:
    promedio = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].mean()
    desviacion = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].std()
    print "El promedio para: "+str(amb)+" ambientes es: "+str(promedio)+" y su desviacion estandar es :"+str(desviacion)


In [ ]:
#P/ cada cantidad de ambientes miro su superficie total, superficie cubierta y precio en u$d 
#y elimino las cosas raras

In [ ]:
df = data_2017_zo_zs_caba.copy()

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 60 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 1) | ((df["rooms"] == 1) & (df["surface_covered_in_m2"] < 61))]

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].max()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df[(df["rooms"] == 1) & (df["price_aprox_usd"] == 6809000)]["description"].values[0]

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 2) | ((df["rooms"] == 2) & (df["surface_covered_in_m2"] < 142))]

In [ ]:
len(df)

In [ ]:
df[df["rooms"] == 3]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 3) | ((df["rooms"] == 3) & (df["surface_covered_in_m2"] < 302))]

In [ ]:
df[df["rooms"] == 4]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 4) | ((df["rooms"] == 4) & (df["surface_covered_in_m2"] < 405))]

In [ ]:
df[df["rooms"] == 8]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 6) | ((df["rooms"] == 6) & (df["surface_covered_in_m2"] < 1300))]

In [ ]:
#Los que tengan mas de 8 ambientes, los pongo como NaN
def filter_ambientes(row):
    ambientes = row["rooms"]
    if ambientes > 8:
        return np.nan
    elif ambientes == 2.5:
        return 3 #Son muy pocos estos dos 
    elif ambientes == 3.5:
        return 4 
    else:
        return ambientes

In [ ]:
df["rooms"] = df.apply(lambda row: filter_ambientes(row), axis = 1)

In [ ]:
df["rooms"].value_counts()

In [ ]:
df["surface_total_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
df2 = df.copy()
len(df2)

In [ ]:
df = df2.copy()
df2 = df.copy()

In [ ]:
df["surface_covered_in_m2"].isnull().sum()

In [ ]:
def replace_cubierta_nan(row):
    cubierta = row["surface_covered_in_m2"]
    if pd.isnull(cubierta):
        return np.nan
    elif cubierta < 20:
        return np.nan
    elif not(pd.isnull(row["rooms"])):
        rooms = row["rooms"]
        if rooms == 4 and cubierta < 30:
            return np.nan
        elif rooms == 5 and cubierta < 35:
            return np.nan
        elif rooms == 6 and cubierta < 40:
            return np.nan
        elif rooms == 7 and cubierta < 45:
            return np.nan
        elif rooms == 8 and cubierta < 59:
            return np.nan
        else:
            return row["surface_covered_in_m2"]
    else:
        return row["surface_covered_in_m2"]

In [ ]:
def replace_total(row):
    sup_total = row["surface_total_in_m2"]
    if sup_total < 20:
        return np.nan
    else:
        return row["surface_total_in_m2"]

In [ ]:
df["surface_total_in_m2"] = df.apply(lambda row: replace_total(row), axis = 1)

In [ ]:
df2["surface_total_in_m2"].isnull().sum()

In [ ]:
df["surface_covered_in_m2"] = df.apply(lambda row: replace_cubierta_nan(row), axis = 1)

In [ ]:
df["surface_covered_in_m2"].isnull().sum()

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].max()

In [ ]:
df.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)

In [ ]:
#Para cada nulo de rooms, le pongo el promedio de los q tengan su cubierta y total, y para cada
#nulo de cubierta, lo mismo
df.isnull().sum()

In [ ]:
def fix_nan_rooms(row, df):
    if not(pd.isnull(row["rooms"])):
        return row["rooms"] #Si ya tenia la cantidad de rooms, esto esta bien
    
    sup_cub = row["surface_covered_in_m2"]
    avg_rooms = np.nan
    dif = 0
    while pd.isnull(avg_rooms):    
        lim_inf = sup_cub - dif
        lim_sup = sup_cub + dif
        act_vecindario = df[df["surface_covered_in_m2"] < lim_sup]
        act_vecindario = df[df["surface_covered_in_m2"] > lim_inf]
        print len(act_vecindario)
        avg_rooms = act_vecindario["rooms"].mean()
        dif += 5
    print "FOUND "+str(avg_rooms)
        

In [ ]:
not_nan_rooms = df[~df["rooms"].isnull()]

In [ ]:
not_nan_rooms["surface_covered_in_m2"].isnull().sum()

In [ ]:
df.corr()

In [ ]:
#Hay una correlacion masomenos alta entre la superficie cubierta y la total, para rellenar la superficie
#cubierta que queda lo hago segun la total

In [ ]:
df2 = df.copy()

In [ ]:
#A los que tengan una superficie total < a la cubierta: Les asigno la cubierta
def fix_surface_total_menor_cubierta(row):
    if pd.isnull(row["surface_total_in_m2"]) or pd.isnull(row["surface_covered_in_m2"]):
        return row["surface_total_in_m2"]
    elif row["surface_total_in_m2"] < row["surface_covered_in_m2"]:
        return row["surface_covered_in_m2"]
    else:
        return row["surface_total_in_m2"]

In [ ]:
df.corr()

In [ ]:
df["surface_total_in_m2"] = df.apply(lambda row: fix_surface_total_menor_cubierta(row), axis=1)

In [ ]:
#Hay alguno con superficie total muy grande?
df["surface_total_in_m2"].max()

In [ ]:
df[df["surface_total_in_m2"] == df["surface_total_in_m2"].max()]["description"].values[0]

In [ ]:
#Lo elimino
df[df["surface_total_in_m2"] == df["surface_total_in_m2"].max()]

In [ ]:
df.sort_values(by="surface_total_in_m2", ascending = False)

In [ ]:
df["surface_total_in_m2"].value_counts().sort_index(ascending = False)

In [ ]:
df[df["surface_total_in_m2"] > 2000]["price_aprox_usd"].mean()

In [ ]:
#Elimino propiedades que tengan una superficie total muy alta y un precio muy bajo

In [ ]:
len(df[(df["surface_total_in_m2"] < 2000) | ((df["surface_total_in_m2"] >= 2000) & df["price_aprox_usd"] > 1000000)])

In [ ]:
df = df[(df["surface_total_in_m2"] < 2000) | ((df["surface_total_in_m2"] >= 2000) & df["price_aprox_usd"] > 1000000)]

In [ ]:
df.corr()

In [ ]:
df["surface_total_in_m2"].max()

In [ ]:
saved_results = {}

In [ ]:
#A partir de la superficie total empiezo a arreglar la sup. cubierta
def fill_superficie_cubierta_con_total(row, df):
    
    if not(pd.isnull(row["surface_covered_in_m2"])):
        return row["surface_covered_in_m2"]
    sup_total = row["surface_total_in_m2"]
    avg_cubierta = np.nan
    dif = 0
    if sup_total in saved_results:
        return saved_results[sup_total]
    while pd.isnull(avg_cubierta):    
        lim_inf = sup_total - dif
        lim_sup = sup_total + dif
        act_vecindario = df[df["surface_total_in_m2"] < lim_sup]
        act_vecindario = act_vecindario[act_vecindario["surface_total_in_m2"] > lim_inf]
        avg_cubierta = act_vecindario["surface_covered_in_m2"].mean()
        dif += 1
        if (dif > 20): #para que sea mas rapido
            dif += 20
    #Guardo el resultado 
    saved_results[row["surface_total_in_m2"]] = avg_cubierta
    return avg_cubierta

In [ ]:
df["surface_covered_in_m2"]= df.apply(lambda row: fill_superficie_cubierta_con_total(row, df), axis = 1)

In [ ]:
print saved_results

In [ ]:
#Por ahora dejamos las propiedades con un valor muy alto, de ultima despues probamos eliminarlas

In [ ]:
saved_surface_total_rooms = {}

In [ ]:
def fix_nan_rooms(row, df):
    if not(pd.isnull(row["rooms"])):
        return row["rooms"] #Si ya tenia la cantidad de rooms, esto esta bien
    
    sup_cub = row["surface_covered_in_m2"]
    if sup_cub in saved_surface_total_rooms:
        return saved_surface_total_rooms[sup_cub]
    avg_rooms = np.nan
    dif = 0
    while pd.isnull(avg_rooms):    
        lim_inf = sup_cub - dif
        lim_sup = sup_cub + dif
        act_vecindario = df[df["surface_covered_in_m2"] < lim_sup]
        act_vecindario = df[df["surface_covered_in_m2"] > lim_inf]
        avg_rooms = act_vecindario["rooms"].mean()
        dif += 5
    saved_surface_total_rooms[sup_cub] = avg_rooms
    return avg_rooms
        

In [ ]:
df2 = df.copy()

In [ ]:
df["rooms"] = df.apply(lambda row: fix_nan_rooms(row,df), axis = 1)

In [ ]:
df.corr()

In [ ]:
#Es una casa de 12 millones de dolares con una descripcion muy pobre, la elimino
#Es complicado considerar un vecindario para expensas, a aquella la eliminamos. con floor podemos 
#rellenar con 0's. Ademas en expensas no se ve una correlacion

In [ ]:
df["floor"].fillna(0, inplace = True)

In [ ]:
df.drop(labels = "expenses", axis = 1, inplace = True)

In [ ]:
#A las que tengan place_name nulo le asigno su state_name
def fix_null_place_name(row):
    if pd.isnull(row["place_name"]):
        return row["state_name"]
    return row["place_name"]

In [ ]:
df["place_name"] = df.apply(lambda row: fix_null_place_name(row), axis = 1)

In [ ]:
len(df[df["lat-lon"].isnull()]["place_name"].value_counts().index)

In [ ]:
df["lat-lon"].values[0]

In [ ]:
#A los de lat-lon nula, busco entre los que tengan el mismo place name y le asigno algun lat lon 
#de ellos
saved_lat_lons = {}
def fix_lat_lon(row, df):
    if not(pd.isnull(row["lat-lon"])):
        return row["lat-lon"]
    place = row["place_name"]
    if place in saved_lat_lons:
        print "FOUND AT SAVED!"
        return saved_lat_lons[place]
    same_place = df[df["place_name"] == place]
    same_place_w_lat_lon = same_place[~same_place["lat-lon"].isnull()]
    try:
        final_pos = same_place_w_lat_lon["lat-lon"].values[0]
    except:
        return np.nan
    saved_lat_lons[place] = final_pos
    return final_pos

In [ ]:
df["lat-lon"] = df.apply(lambda row: fix_lat_lon(row,df), axis = 1)

In [ ]:
#Quedaron 11 nulos, los busco a mano

In [ ]:
df["lat-lon"] = df.apply(lambda row: row["lat-lon"] if not(pd.isnull(row["lat-lon"])) else "-34.800986,-58.218988", axis = 1)

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv("training_set_filling_nans_con_vecinos.csv", compression ="gzip", index = False)

In [ ]:
test_set_inicial = pd.read_csv("LO IMPORTANTE/properati_testing_noprice_fixed.csv",compression ="gzip", low_memory = False)

In [ ]:
#primero pongo en nuls las cosas raras


In [ ]:
test_set_inicial["rooms"] = test_set_inicial.apply(lambda row: filter_ambientes(row), axis = 1)

In [ ]:
test_set_inicial["rooms"].value_counts()

In [ ]:
len(test_set_inicial)

In [ ]:
test_set_inicial["surface_covered_in_m2"] = test_set_inicial.apply(lambda row: replace_cubierta_nan(row), axis = 1)

In [ ]:
test_set_inicial["surface_total_in_m2"] = test_set_inicial.apply(lambda row: replace_total(row), axis = 1)

In [ ]:
#Ademas tengo que chequear monoambientes o 2, 3, .. con superficie cubierta ridicula 

In [ ]:
test_set_inicial[(test_set_inicial["rooms"] == 1) & (test_set_inicial["surface_covered_in_m2"] == 1900)]["description"].values[0]

In [ ]:
test_set_inicial[(test_set_inicial["rooms"] == 1) & (test_set_inicial["surface_covered_in_m2"] > 100)]

In [ ]:
a["rooms"].value_counts()